# 3rd Homework of ADM

### Group members

#### 1. Mehrdad Hassanzadeh, 1961575, hassanzadeh.1961575@studenti.uniroma1.it
#### 2. Vahid Ghanbarizadeh, 2002527, ghanbarizdehv@gmail.com
#### 3. Andrea Giordano , 1871786, giordano.1871786@studenti.uniroma1.it

# Importing useful packages

In [1]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import os
from pathlib import Path
import re
import multiprocessing as mp
from math import ceil
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import json
import pandas as pd
from math import log
from copy import deepcopy
from heapq import heappop, heappush  

# 1. Data collection

We are asked to collect some information about the anime in a spcific website. Here is the link to that site. 'https://myanimelist.net/topanime.php'

Here we send a request to get the content of a specific webpage. We wanted the content of the main page of the TopAnime website. 

In [3]:
TopAnime = requests.get('https://myanimelist.net/topanime.php')

Now we check if we have successfully received the content of the desired webpage or not. 

In [4]:
TopAnime

<Response [200]>

As we have the code '200' as the status of the reponse, so it means that we have successfully received the desired page. 

Then we take a brief look at the content of this page. 

In [100]:
#TopAnime.content

As we can see in the output, we have the HTML code of the webpage. Now we should parse this HTML code in order to get the data we want. To do so, we will be using **BeautifulSoap** library which is designed to parse HTML codes. 

In [7]:
TopAnimeSoup = BeautifulSoup(TopAnime.content, 'html.parser')

Now we look at the produced HTML code of the webpage in nice and more readable format using BeautifulSoup. 

In [101]:
#print(TopAnimeSoup.prettify())

# 1.1. Get the list of animes

### Steps to extract the url for just one anime. 

Here we will go through the required steps to extract the url and the name of just the first anime in the list. We can then expand this idea to all the anime in the page. 

### 1.1.1 Getting all the rows (anime) in a page

After chekcing the HTML code, we saw that the information related to each anime is stored in a table which its class is "top-ranking-table" and the animes' information are stored in trs of this table. 

Let's take a look at how many tables we have in the webpage. 

In [9]:
len(list(TopAnimeSoup.find_all('table')))

1

As we have only one table in the webpage, so every tr that we have in the webpage belongs to this table. 

Here we will take a look at how many tr tags we have in the webpage. 

In [111]:
#print(list(TopAnimeSoup.find_all('tr')))
#len(list(TopAnimeSoup.find_all('tr')))

We know that in each page, we have the information related to 50 animes. But why we have 51 tr tags here? <br/>
Because the first row corresponds to the name of table's column and the rest store the inramtion related to each anime. 

So in order to get the rows which contain the information of each anime, we should go through the tr tags that we have in the webpage except the first one which contains the information of the columns' name of of the table. 

In [12]:
Rows = list(TopAnimeSoup.find_all('tr'))[1:]
len(Rows)

50

Here we have all the information of the anime in a specific page. As we know the number of anime in each page is 50. 

### 1.1.2 Extracting the name and url of an anime

Now we should get the name and the url correspond to each anime. We found out that this information can be found in 'a' tag of each row which its class name is "hoverinfo_trigger fl-l ml12 mr8" and is included in the second 'td' tag of each 'tr'.  

We get all the 'td' tags of the second 'tr' the first 'tr' tag contains the columns' name. 

In [14]:
Tds = Rows[0].find_all('td')

Then we will go to the second 'td' tag's information. The first one contains just the ranking number. 

In [15]:
SecondTD = Tds[1]

Then inside this 'td' tag we look for the 'a' tag which its class is "hoverinfo_trigger fl-l ml12 mr8"

In [107]:
TagA = SecondTD.find('class' == "hoverinfo_trigger fl-l ml12 mr8")
TagA

<a class="hoverinfo_trigger fl-l ml12 mr8" href="https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood" id="#area5114" rel="#info5114">
<img alt="Anime: Fullmetal Alchemist: Brotherhood" border="0" class="lazyload" data-src="https://cdn.myanimelist.net/r/50x70/images/anime/1223/96541.jpg?s=faffcb677a5eacd17bf761edd78bfb3f" data-srcset="https://cdn.myanimelist.net/r/50x70/images/anime/1223/96541.jpg?s=faffcb677a5eacd17bf761edd78bfb3f 1x, https://cdn.myanimelist.net/r/100x140/images/anime/1223/96541.jpg?s=0c3b98cf4905422c00981025cd20d271 2x" height="70" width="50">
</img></a>

The url of an anime is the value of 'href' property of this tag. Let's take a look at it. 

In [108]:
URL = TagA['href']
URL

'https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood'

Here to extract the name of the anime we have two options. 1. To split the 'href' and get the last value of it. 2. Get the value of 'alt' property of the 'img' tag in the 'a' tag. Here we will go for the second approach. 

In [112]:
Image = TagA.find('img')
AnimeName = Image['alt'].replace('Anime: ', '')
AnimeName

'Fullmetal Alchemist: Brotherhood'

### 1.1.3 Getting the information for all anime in a page

Now here we want to get the name and url of all the animes in this specific page. 

In [113]:
# Dictionary to store the url or each anime in the page 
MyDict = defaultdict(str)

# Going through all the rows in the page
for Row in Rows: 
    
    # Take the second column of the row
    TDs = Row.find_all('td')
    
    # Take the tag a which consists the anime name and the url of that anime
    TagA = TDs[1].find('class' == "hoverinfo_trigger fl-l ml12 mr8")
    
    # Extract the anime name and url of the anime
    AnimeName, URL = TagA.find('img')['alt'].replace('Anime: ', ''), TagA['href']
    
    # Store the url of the anime in the dictionary
    MyDict[AnimeName] = URL

Now we will check the information for five animes. 

In [115]:
for anime in list(MyDict.keys())[:5]:
    print('Name of anime: ' + anime+'\nURL: ' + MyDict[anime], end = '\n\n')

Name of anime: Fullmetal Alchemist: Brotherhood
URL: https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood

Name of anime: Gintama°
URL: https://myanimelist.net/anime/28977/Gintama°

Name of anime: Shingeki no Kyojin Season 3 Part 2
URL: https://myanimelist.net/anime/38524/Shingeki_no_Kyojin_Season_3_Part_2

Name of anime: Steins;Gate
URL: https://myanimelist.net/anime/9253/Steins_Gate

Name of anime: Fruits Basket: The Final
URL: https://myanimelist.net/anime/42938/Fruits_Basket__The_Final



Here we want to check if we have all the 50 animes' URL in the dictionay. 

In [118]:
print('Number of animes ', len(MyDict))

Number of animes  50


# 1.2 Crawl animes

### 1.2.1 Function Crawl urls in a webpage

In order to have more readable implementation we will write a function which receives the URL of the webpage that we want to scrap and writes the urls to the URLs.txt file. 

In [149]:
def GetAnimeInfo(webpage):
    
    # Opening the file that we want to write our information in 
    File = open('URLs.txt', mode = 'a')
    
    # Send a request to get the webpage
    Request = requests.get(webpage)
    
    # Parse the HTML code of the webpage
    TopAnimeSoup = BeautifulSoup(Request.content, 'html.parser')
    
    # Take all the tr tags in the webpage which contain all the anime information in the given webpage
    Rows = TopAnimeSoup.find_all('tr')
    
    # Going throguh all the anime 
    for Row in Rows[1:]: 
        
        # Get the td tags of each anime 
        TDs = Row.find_all('td')
        
        # Take the tag A which its class is equal to 'hoverinfo_trigger fl-l ml12 mr8'
        TagA = TDs[1].find('class' == "hoverinfo_trigger fl-l ml12 mr8")
        
        # Extract the URL of the anime
        URL = TagA['href']
        
        # Write the URL to the file
        File.write(URL+'\n')
        
    # Close the file we have just written it
    File.close()

Now at each time we should pass the function the URL of the webpage that we want to scrap. 

After checking the URL of the next pages, we understood that there is a pattern in URL of the pages. <br/>
For example the 2nd webpage's URL is 'https://myanimelist.net/topanime.php?limit=50' and we can see the only difference between this URL and the main page URL (which is 'https://myanimelist.net/topanime.php') is that there is '?limit=50' string at the end. <br/><br/>
* So we can use this pattern the find the next pages URL. 

### 1.2.2 Main Function to extract all the URLs

In [148]:
# This is the root webpage of the website
MainPageURL = 'https://myanimelist.net/topanime.php'

# We go through first 400 pages 
for i in range(400):
    
    # This is the address that we want for the first page
    if i == 0:
        GetAnimeInfo(MainPageURL)
        
    # For the next pages, we should use the address of the webpage and put the limit 
    # at the end to reach to the next pages
    else:
        GetAnimeInfo(MainPageURL+'?limit='+str(50*i))
    

### 1.2.3 Function Get HTML code

Now we want to download the HTML file for each anime. <br/> 
In order to do this we are going to write a function that passes a number of URLs which their HTML file should be downloaded and stored in a folder.<br/>
These URLS are given in a list of list which correspond to a number of webpages in the website and a number of anime's URL which are in a specific webpage. 

* **Note:** We will download the HTML codes in parallel by using all the processors in the system. 

As this function will be called from all the processors in the system, we should find a way so that all the processors can download their own given pages' HTML code. 

In [161]:
# The parameters of the function: 

# URLS: A list of a list which each element in the first list is the pages and each element in the list 
# of each page is the anime's URL in that page that we should get their HTML code. 

# GroupIndex: Specifies which processor is calling this function 

# NumPageGroup: Number of the pages that was given to each processor

# Start: Start the downloading from which page? Start = 50 -> Starting extracting the HTML code of the anime 
# in the page 50

def GetHTML(URLS, GroupIndex, NumPageGroup, Start):
    
    # Going through each page
    for index, ListURLs in enumerate(URLS):
        
        # If the page number that we are considering at this moment lower than Start, we will ignore it
        if index < Start:
            continue
            
        # For each page we create a new folder to store its anime HTML code in it. 
        FolderName = 'page' + str(GroupIndex*NumPageGroup + index +1)
        Path = os.path.join(os.getcwd(),'HTMLS', FolderName)
        os.mkdir(Path)
        
        # Iterating over all the anime URL in a specific page
        for i, url in enumerate(ListURLs):
            
            # Request to get the webpage of that anime's URL
            AnimePage = requests.get(url)
            
            # Get the HTML code of that webpage
            AnimePageSoup = BeautifulSoup(AnimePage.content, 'html.parser')
            
            # Create a html file to store the HTML code of the webpage in it
            file = open(Path + '/' + 'anime_' + str(i) + '.html',"w")
            file.write(str(AnimePageSoup))
            file.close()    

### 1.2.4 Parallelizing the Crawling process

So first we check how many processors we have in the system:

In [2]:
print("The number of processors:", mp.cpu_count())

The number of processors: 4


So in this case we should distribute our URLS over these processors. We read all the URLs that we have stored in the 'URLs.txt' file.

In [2]:
AllURLs = []
with open('URLs.txt') as f:
    for row in f:
        AllURLs.append(row.strip())

Now we want to check how many URLs we have:

In [45]:
print('The total number of URLs is:', len(AllURLs))

The total number of URLs is: 19128


* **Note:** We know that we should store all the anime's webpage HTML code in a single folder for the ones that have been appeared in the same wepbage, so all of these URLs should be entirely given to a processor. 

* **Note:** We know that we extracted the URLs in order, so if we take the URLs 50 by 50 we will have the anime URLs that are in the same page. 

Now we group each 50 URLs to construct the pages. 

In [46]:
EachPageURLs = [AllURLs[i*50:(i+1)*50] for i in range(len(AllURLs)//50+1)]

Here we will take a look at how many pages that we have here: 

In [47]:
print("Number of webpages in the website:", len(EachPageURLs))

Number of webpages in the website: 383


Now we check how many pages should be given to each of the processors. 

In [359]:
NumOfPage = ceil(ceil(len(AllURLs)/50)/mp.cpu_count())
print('Number of pages to be given to each processor: ', NumOfPage)

Number of pages to be given to each processor:  96


* Each processor is responsible to download the anime's HTML code that are in 96 pages. 

Now we group the pages that should be given to each of the processors. 

In [9]:
GroupPage = [EachPageURLs[i*NumOfPage:(i+1)*NumOfPage] for i in range(mp.cpu_count())]

Let's check the number of pages in each group:

In [10]:
print(list(map(len , GroupPage)))

[96, 96, 96, 95]


As we can see all the processors have been assigned 96 pages to download except the last one. We remind that we have 383 page which couldn't be distributed evenly between all the processors. 

### Downloading HTMLs in parallel

As we saw before, we have 4 processors in the system and we want to distribute the work among these 4 processors.<br/>
We will give a subsets of pages to each of these processors to download their anime HTMl code. 

In [196]:
# How many processors we want to pool handle 
pool = mp.Pool(mp.cpu_count())

# Call the GetHTML function for each of the processors given their assigned pages
results = [pool.apply_async(GetHTML, args = (GroupPage[i],i, NumOfPage, 5)) for i in range(mp.cpu_count())]
pool.close()

# 1.3 Parse downloaded pages

Now in this question we should produce a .tsv file for each anime which contains some information extracted from their webpage. 

In order to do this, first we check how we can get the information for just one anime and then we will expand the idea to other anime that we have. 

The information that we should extract for each anime are: 

1- **Anime Name** (to save as animeTitle): String <br/>
2- **Anime Type** (to save as animeType): String<br/>
3- **Number of episode** (to save as animeNumEpisode): Integer<br/>
4- **Release and End Dates of anime** (to save as releaseDate and endDate): Convert both release and end date into datetime format.<br/>
5- **Number of members** (to save as animeNumMembers): Integer<br/>
6- **Score** (to save as animeScore): Float<br/>
7- **Users** (to save as animeUsers): Integer<br/>
8- **Rank** (to save as animeRank): Integer<br/>
9- **Popularity** (to save as animePopularity): Integer<br/>
10- **Synopsis** (to save as animeDescription): String<br/>
11- **Related Anime** (to save as animeRelated): Extract all the related animes, but only keep unique values and those that have a hyperlink associated to them. List of strings.<br/>
12- **Characters** (to save as animeCharacters): List of strings.<br/>
13- **Voices** (to save as animeVoices): List of strings<br/>
14- **Staff** (to save as animeStaff): Include the staff name and their responsibility/task in a list of lists.<br/>

* For getting each of the information that we need, we write a function to have a better understanding of how we should extract each of these information. 

### 1.3.1 Function AnimeName

We know that the name of the anime can be extracted from the title of the webpage. So we send the html code of the webpage to this function, and we will return the name of the anime extracted from the title of the page. 

In [59]:
def GetAnimeName(webpage):
    return webpage.title.getText().strip().replace(' - MyAnimeList.net', '')

### 1.3.2 Function GetAnimeType

This information can be extracted from the values stored in a specific 'div' tag. In order do this we will send that specific tag to this function and receive the type of the anime. 

In [60]:
def GetAnimeType(Tag):
    Temp = Tag.getText().split()
    return Temp[-1] if len(Temp) >1 else ""

### 1.3.3 Function Episode

This information can be extracted from the values stored in a specific 'div' tag. In order do this we will send that specific tag to this function and receive the type of the anime. 

In [61]:
def GetNumOfEpisode(Tag):
    Temp = Tag.getText().strip().split()[-1]
    return int(Temp) if Temp.isdigit() else ''

### 1.3.4 Function DateTime

This information can be extracted from the values stored in a specific 'div' tag. In order do this we will send that specific tag to this function and receive the type of the anime. <br/><br/>
* Some of the animes may have just the release date. So we should be careful about this. 

In [63]:
def GetDates(Tag):
    Release, End = '', ''
    Temp = Tag.getText().strip().replace('Aired:\n  ', '').split('to')
    Release = Temp[0] if len(Temp) else ''
    End = Temp[1] if len(Temp) == 2 else ''
    return Release, End

### 1.3.5 Function Members

This information can be extracted from the values stored in a specific 'div' tag. In order do this we will send that specific tag to this function and receive the type of the anime.

In [64]:
def GetMembers(Tag):
    return int(Tag.getText().replace('\n', '').split()[1].replace(',', ''))

### 1.3.6 Function ScoreAndUsers

This information can be extracted from the values stored in a specific 'div' tag. In order do this we will send that specific tag to this function and receive the type of the anime.

In [65]:
def GetScoreAndUsers(Tag):
    Rank = re.findall('[0-9|,]+ users', str(Tag))
    RankValue = int(Rank[0].split()[0].replace(',', '')) if len(Rank) else ''
    Score = re.findall('Score:[0-9|.]+', Tag.getText().replace("\n", ''))
    ScoreValue = float(Score[0].split(":")[1]) if len(Score) else ''
    return ScoreValue, RankValue

### 1.3.7 Function Rank

This information can be extracted from the values stored in a specific 'div' tag. In order do this we will send that specific tag to this function and receive the type of the anime.

In [66]:
def GetRank(Tag):
    Temp = re.findall('#[0-9|,]+', str(Tag))
    return int(Temp[0][1:]) if len(Temp) else ''

### 1.3.8 Function Popularity

This information can be extracted from the values stored in a specific 'div' tag. In order do this we will send that specific tag to this function and receive the type of the anime.

In [67]:
def GetPopularity(Tag):
    Temp = re.findall('#[0-9|,]+', str(Tag))
    return int(Temp[0][1:]) if len(Temp) else ''

### 1.3.9 Function Synopsis

In order to get the description of the anime, there is a specific tag which can be easily identified by its 'itemprop' property. So we will give the whole html to this function and get back the description. 

In [68]:
def GetSynopsis(webpage):
    Temp = webpage.find('p', {'itemprop': "description"})
    if not Temp:
        return ''
    Temp = Temp.getText().replace('\n', '')
    return Temp

### 1.3.10 Function Related Anime

To get the related animes, there is a table which its 'class' is equal to 'anime_detail_related_anime'. So to get the table we should give the function the whole html file to this function. 

In [69]:
def GetRelatedAnime(webpage):
    Related = webpage.find('table', {'class': "anime_detail_related_anime"})
    if not Related:
        return ""
    Related = Related.find_all('a')
    UniqueRelated = set(i.getText() for i in Related)
    return list(UniqueRelated)

### 1.3.11 Function Characters

We can find this information in tag 'a' that are included in a div which is its class is equal to 'detail-characters-list clearfix'. When we go for the first div that have this feature we will be received the table which is the characters and their original voices is there. 

In [70]:
def GetCharacters(webpage):
    Characters = []
    Tags = webpage.find_all('div', {'class': "detail-characters-list clearfix"})
    for tag in Tags:
        if str(tag).count('character') >1:
            
            # We can get the name of the character in the 'href' value of the tag 'a' property
            AllTagA = tag.find_all('a')
            
            # Filter the hrefs to of the characters
            CharactersHrefs = list(set([i['href'] for i in AllTagA if 'character' in i['href']]))
            
            # Filter the names of the characters
            Characters = [i.split('/')[-1].replace('_', ' ') for i in CharactersHrefs]
    return Characters

### 1.3.12 Function Voices 

With the same approach as previous, now we just extract the voices. 

In [71]:
def GetVoices(webpage):
    Voices = []
    Tags = webpage.find_all('div', {'class': "detail-characters-list clearfix"})
    for tag in Tags:
        if str(tag).count('character') > 1:
            
            # We can get the name of the person in charge for the voice in the 'href' value in tag 'a' property
            AllTagA = tag.find_all('a')
            
            # Filter the hrefs for voices
            VoicesHrefs = list(set([i['href'] for i in AllTagA if 'people' in i['href']]))
            
            # Filter the names of the people
            Voices = [i.split('/')[-1].replace('_', ' ') for i in VoicesHrefs]
    return Voices

### 1.3.13 Function Staff

Here in this function we will look for the other div tha its 'class' is equal to 'detail-characters-list clearfix'. Then the name of the staff can be extracted from 'img' tags and also their duties can be found in the 'small' tags which are in the same 'row' as their image. 

In [97]:
def GetStaff(webpage):
    StaffDuty = []
    Tags = webpage.find_all('div', {'class': "detail-characters-list clearfix"})
    for tag in Tags:
        if str(tag).count('character') == 1:
            Staff =  tag.find_all('tr')
            for i in Staff:
                NewStaff = [i.find('a')['href'].split('/')[-1].replace('_', ' ')] # Extracting the name of the staff
                Duties = list(i.find('small').getText().split(',')) # Getting the duties of the staff
                NewStaff += [i.strip() for i in Duties]
                StaffDuty.append(NewStaff)
    return StaffDuty

### 1.3.14 Function Write to TSV

With the help of this function, we can write the information that we extracted from a page to its corresponded .tsv file. 

In [73]:
def WriteToTSV(AnimeInfo, Path):
    
    # Open the corresponded file  
    File = open(Path, mode = 'w')
    
    # Writing the header. 
    File.write("\t".join(Keys))
    File.write('\n')
    
    # Going through all the information for that specific anime
    for i in AnimeInfo:
        
        # Get the string version of that value
        STR = str(AnimeInfo[i])
        
        # In case an information was missing, we should put '' in the file
        if STR == '[]':
            STR = ''
        
        # Write the value of the information into the file
        File.write(STR + ('\t' if i!='animeStaff' else "" ))
    File.close()    

### 1.3.15 Function Extract

With the help of this function we want to extract the desired information from all the .html files that we have. 

Here we used some key words so we can easily get the needed information which are in some tags that share the same class. 

In [74]:
Info = ['Type:', 'Episodes:', 'Aired:', 'Members:','Score:','Ranked:', 'Popularity:']

Here we define the default datastructure which is a dictionary to store the inforamtion of each page. 

In [75]:
Keys = ['animeTitle', 'animeType', 'animeNumEpisode', 'releaseDate', 'endDate', 'animeNumMembers'
        , 'animeScore', 'animeUsers', 'animeRank', 'animePopularity', 'animeDescription', 'animeRelated',
        'animeCharacters', 'animeVoices', 'animeStaff']

# Empty instance for storing the information for each anime
MyAnimeInfo = {i:"" for i in Keys} 

Here we will go through each HTML file that we have just downloaded. Then we extract the infomration that we need from the HTML file and then store them in a .tsv file. 

* **Note:** as we wanted to avoid each processor override the datastructure of another processor we gave each processor its own private datastructure. That is the usage of exec functions. 

* exec function: Given a string it will turn that string into a python code and run the code. In this case we gave this possibility to each processor to work on its own datastructure. 

In [78]:
def FunctionExtract(ListNumPage, ProcesserNum):
    
    # Going through all the pages
    for i in ListNumPage:
        
        # Going through all the anime in each page
        for j in range((50 if i!= 383 else 28)):
            
            # Get the copy of the default datastructures
            exec('NewAnime'+str(ProcesserNum)+'= MyAnimeInfo.copy()')
            
            # Reading the .html file of a specific anime
            Path = '/home/mehrdad/ADM-HW3/HTMLS/page'
            File = open(Path + str(i) + '/anime_' + str(j) +'.html', mode = 'r')
            
            # Get the parsed HTML code of the webpage of the anime 
            Webpage = BeautifulSoup(File.read(), 'html.parser')
            
            # Extracting some of the information that have been stored in the tags that have the 
            # same value as the class
            for div in Webpage.find_all('div', {'class':"spaceit_pad"}):
                if Info[0] in str(div):
                    exec('NewAnime'+ str(ProcesserNum) +"['animeType'] = GetAnimeType(div)")
                elif Info[1] in str(div):
                    exec('NewAnime'+ str(ProcesserNum) +"['animeNumEpisode'] = GetNumOfEpisode(div)")
                elif Info[2] in str(div):
                    exec('NewAnime'+ str(ProcesserNum) +"['releaseDate'] = GetDates(div)[0]")
                    exec('NewAnime'+ str(ProcesserNum) +"['endDate'] = GetDates(div)[1]")
                elif Info[3] in str(div):
                    exec('NewAnime'+ str(ProcesserNum) +"['animeNumMembers'] = GetMembers(div)")
                elif Info[4] in str(div):
                    exec('NewAnime'+ str(ProcesserNum) +"['animeScore'] = GetScoreAndUsers(div)[0]")
                    exec('NewAnime'+ str(ProcesserNum) +"['animeUsers'] = GetScoreAndUsers(div)[1]")
                elif Info[5] in str(div):
                    exec('NewAnime'+ str(ProcesserNum) +"['animeRank'] = GetRank(div)")
                elif Info[6] in str(div):
                    exec('NewAnime'+ str(ProcesserNum) +"['animePopularity'] = GetPopularity(div)")
            
            # Extracting the other information 
            exec('NewAnime'+ str(ProcesserNum) +"['animeTitle'] = GetAnimeName(Webpage)")
            exec('NewAnime'+ str(ProcesserNum) +"['animeDescription'] = GetSynopsis(Webpage)")
            exec('NewAnime'+ str(ProcesserNum) +"['animeRelated'] = GetRelatedAnime(Webpage)")
            exec('NewAnime'+ str(ProcesserNum) +"['animeCharacters'] = GetCharacters(Webpage)")
            exec('NewAnime'+ str(ProcesserNum) +"['animeVoices'] = GetVoices(Webpage)")
            exec('NewAnime'+ str(ProcesserNum) +"['animeStaff'] = GetStaff(Webpage)")
            
            TSVPath = Path + str(i) + '/anime_' + str(j) +'.tsv'
            exec('WriteToTSV(NewAnime'+ str(ProcesserNum)+',TSVPath)')

### Creating the .tsv files in parallel. 

* **In order to speed up the process, we will distribute the work among the available CPUs.**

We should give a subset of pages to each CPU to make its animes' .tsv file. <br/>
Here we will group the page numbers that should be given to each processor. 

In [79]:
RangePage = list(range(1, len(EachPageURLs) + 1))
PageNums = [RangePage[i * NumOfPage:(i+1) * NumOfPage] for i in range(mp.cpu_count())]

Call the function for each CPU give a susbset of .html files. 

In [99]:
pool = mp.Pool(mp.cpu_count())
results = [pool.apply_async(FunctionExtract, args = (PageNums[i],i)) for i in range(mp.cpu_count())]
pool.close()

# 2. Search Engine


Here we are asked to build a search engine which given a query, will give back the documents that are similar to the given query. 

### 2.0 Pre-processing the information

Here we will pre-process all the information of an anime and store it to another .tsv file which will be name SynopsisPrepAnime_(i).csv which i corresponds to the index number of the page in its page. 

To do pre-processing we have 5 stages and for each stage we will write a function. 

### 2.0.1 Tokenization 

Given a string, we will return the words that are in the given string. 

In [3]:
# Using this functions to extract the terms in each given sentence 
def Tokenization(Sentence):
    return nltk.word_tokenize(Sentence)

### 2.0.2 Lowercasing

Given a list of strings, we will return the same strings but in lower case. 

In [4]:
# Turn all the words that we have in the given list of words to their lowercase
def Lowercasing(Words):
    return [w.lower() for w in Words]

### 2.0.3 StopWordsRemoval

Here we will define a list which contains all the stopwords in English language. 

In [5]:
StopWords = stopwords.words('english')

Given a list of strings, we will remove the stopwords from that strings. 

In [6]:
# We will consider just those terms that are not in stopwords
def StopWordsRemoval(Words):
    return [w for w in Words if w not in StopWords]

### 2.0.4 PunctuationsRemoval

In this function given a list of strings, we will remove the punctuations from those strings. 

In [7]:
# To remove the punctuations, we only will consider the alphabetic letters
def PunctuationsRemoval(Words):
    return [w for w in Words if w.isalpha()]

### 2.0.5 Stemming

In this function, given a list of strings, we try to return back the stem of each string that we have in the list. <br/>
Here we will use 'PorterStemmer' algorithm to do stemming. 

In [8]:
# The stem of each word will be returned 
def Stemming(Words):
    return [PorterStemmer().stem(w) for w in Words]

* Here we are asked to work with the 'Synopsis' of each anime, so we decided pre-process only the synopsis of each anime and not all the information that we extracted. 

* **Note:** For your information, we want to do the pre-process we will distribute the work among the available number of CPUs in the system. Each CPU will be given a subset of pages to do the pre-process for the contained anime information. 

### 2.0.6 Main Function 

In [14]:
def MainFunction(Pages):
    
    # For each page 
    for page in Pages:
        
        # For each anime in a specific page
        for anime in range((50 if page != 383 else 28)):
            
            # Read the '.tsv' file of that specific anime
            Path = '/home/mehrdad/ADM-HW3/HTMLS/page'
            File = open(Path + str(page) + '/anime_' + str(anime) +'.tsv', mode = 'r')
            
            # For each anime we want to extract the synopsis 
            Data = File.read().split('\n')[1].split('\t')[10]
            File.close()
            
            # Given the synopsis of a specific anime to pre-processing stage
            Tokens = Tokenization(Data)
            Lowercase = Lowercasing(Tokens)
            WithoutStop = StopWordsRemoval(Lowercase)
            WithoutPuncs = PunctuationsRemoval(WithoutStop)
            Stems = Stemming(WithoutPuncs)
            
            # Store the result of the pre-processing stage in a '.csv' file for each specific anime
            PreProcessed = open(Path + str(page) + '/anime_' + str(anime) +'_synopsisPrep.csv', mode = 'w')
            PreProcessed.write(",".join(Stems))
            PreProcessed.close()

## 2.1 Conjunctive query

Here we will pre-process the synopsis of each anime. 

Group the pages to be given to the CPUs. 

In [109]:
RangePage = list(range(1, len(EachPageURLs) + 1))
PageNums = [RangePage[i * NumOfPage:(i+1) * NumOfPage] for i in range(mp.cpu_count())]

Pre-processing all the synopsis of animes. 

In [110]:
pool = mp.Pool(mp.cpu_count())
results = [pool.apply_async(MainFunction, args = (PageNums[i],)) for i in range(mp.cpu_count())]
pool.close()

## 2.1.1 Create your index!

Here we will create a file name **'vocabulary.json'** which will map each word that we want to take into account for our analysis to a specific ID. 

We will go through all the pre-processed files that we have created for each anime and extract all the words in them and give each word a specific ID. 

In [11]:
# Dictionary which stores each words in all anime 
MyDict = dict()

# For each page
for i in range(1, 384):
    
    # For each anime
    for j in range((50 if i != 383 else 28)):
        
        # Open the '.csv' file of each specific anime
        Path = '/home/mehrdad/ADM-HW3/HTMLS/page'
        File = open(Path + str(i) + '/anime_' + str(j) +'_synopsisPrep.csv', mode = 'r')
        
        # Get the words in the synopsis of each anime
        DescriptionWord = File.read().split(',')
        
        # For each word in the synopsis we will have an entry in the dictionary
        for term in DescriptionWord:
            MyDict[term] = 0 
            
# Assign each word a unique id
for term_id,term  in enumerate(MyDict.keys()):
    MyDict[term] = term_id

# Writing the resulted information into a .json file so we can easily retrieve them
with open("Vocabulary.json", "w") as write_file:
    json.dump(MyDict, write_file)

### 2.1.1.1 Creating Inverted Indexes

Here we will go through all the preprocessed files and add the documents id to the values of the terms we find in each document. 

We will store the document ids as a combination of page number of anime id (#page, #anime_id).

* We will store our created inverted index into a file named **'InvertedIndex_1'**. 

In [3]:
# A dictionary which have all the terms in the corpus and their term_id
Vocabularies = json.load(open("Vocabulary.json", "r"))

# We will store our inverted index in this dictionary
# Keys: term
# Value: a list of document_id (which is a pair of page and anime index) 
InvertedIndexes = defaultdict(set)

# For each page 
for i in range(1, 384):
    # For each anime 
    for j in range((50 if i != 383 else 28)):
        
        # Read the '.csv' file for each specific anime
        Path = '/home/mehrdad/ADM-HW3/HTMLS/page'
        File = open(Path + str(i) + '/anime_' + str(j) +'_synopsisPrep.csv', mode = 'r')
        
        # Extract the words in the synopsis for each anime
        DescriptionWord = File.read().split(',')
        
        # For all the terms that are in this anime, we will add the document_id to its list
        for term in DescriptionWord:
            InvertedIndexes[Vocabularies[term]].add((i,j))
            
# Sorting the document_ids
for term in InvertedIndexes:
    InvertedIndexes[term] = sorted(InvertedIndexes[term])
    
# We will store these information in a '.json' file so we can easily retrieve them
with open("InvertedIndex_1.json", "w") as write_file:
    json.dump(InvertedIndexes, write_file)

## 2.1.2 Execute the query

Here given a query we want to return the documents that are containing all the words in the given query. 

In order to answer the query, first we should pre-process the query as well. We need to do this as we have pre-processed all the information that we have extracted from the htmls. 

* Here we will write a function which given a string, it will pre-process the string and return the results. 

### 2.1.2.1 Function String Pre-process

In [13]:
# Given a stirng, the string will go to the pre-process stage
def StringPreProcess(string):
    Tokens = Tokenization(string)
    LowerCased = Lowercasing(Tokens)
    WithoutStop = StopWordsRemoval(LowerCased)
    WithoutPunc = PunctuationsRemoval(WithoutStop)
    Stemmed = Stemming(WithoutPunc)
    return Stemmed

### 2.1.2.2 Function token to TermID

This function given a token and the Vocabulary data structure will return back the term_id of that token. If the token is not in the Vocabulary, this function will return None. 

In [14]:
def WordToTermID(Token, Vocabulary):
    return Vocabulary.get(Token)

### 2.1.2.3 Function Extracting Documents

Given a TermID and inverted indexes, this function will return the documents which are containing this specific token. If TermID is equal to 'None', this function will return empty list. 

In [15]:
def ExtractDocs(TermId, InvertedIndex):
    return InvertedIndex.get(str(TermId)) if TermId else []

### 2.1.2.4 Function Query to Documents

This function will process the given query and return back the documents that are containing all the words in the query. 

In [16]:
def QueryToDocuments(Query, Vocabulary, InvertedIndex):
    PreProcessedQuery = StringPreProcess(Query)
    
    # If the query after the pre-process stage was empty, return an empty list
    if not len(PreProcessedQuery):
        return []
    
    # This list will store all the documents in which each term was there. 
    Documents = []

    # Convert each term to its termid
    TermIDs = [WordToTermID(term, Vocabulary) for term in PreProcessedQuery]
    
    # All the documents for each Term
    Documents = [set(map(tuple, ExtractDocs(TermID, InvertedIndex))) for TermID in TermIDs]
    
    # Take the intersection of the documents
    Result = sorted(Documents[0].intersection(*Documents))
    return Result 

### 2.1.2.5 Function Data to Dataframe

This function given the name of the columns and the values for each column will create a dataframe with those information which help to visualize the results in tabular format. 

In [17]:
# Given the column name and the records, will produce a dataframe
def DataToDataFrame(ColumnsName, Records):
    
    # Declare a datafram with specific column name
    Table = pd.DataFrame(columns = ColumnsName)
    
    # For each records 
    for rec in Records:
        
        # We will store the values in each records in a dictionary as for adding a record to the dataframe
        # we should pass a dictionary
        TempDict = dict()
        
        # For each value of the columns of the records
        for col in range(len(ColumnsName)):
            
            # Give the value of that column to the dictionary
            TempDict[ColumnsName[col]] = rec[col]
            
        # Add this new record to the dataframe
        Table = Table.append(TempDict,ignore_index = True )
    display(Table)

### 2.1.2.6 Function Main Answer Query

In [364]:
# Loading Vocabulary
Vocabulary = json.load(open("Vocabulary.json", "r"))

# Loading Inverted Indexes
InvertedIndex = json.load(open("InvertedIndex_1.json", 'r'))

Query = input('Please enter your query here: ')

# Results 
Results = QueryToDocuments(Query, Vocabulary, InvertedIndex)

# To pass the data to show in tabular format 
Columns, Data = ['animeTitle', 'animeDescription', 'URL'], []

# Now here we should iterate over all the results and retrieve the information for each of these anime
for Res in Results:
    Path = '/home/mehrdad/ADM-HW3/HTMLS/page'
    FileURL = open('URLs.txt', mode = 'r')
    FileInfo = open(Path + str(Res[0]) + '/anime_' + str(Res[1]) +'.tsv', mode = 'r')

    Information = FileInfo.read().split('\n')[1].split('\t')
    Title = Information[0]
    Description = Information[10]
    URL = FileURL.read().split('\n')[(Res[0]-1)*50 + Res[1]]
    Data.append([Title, Description, URL])

# Showing the results
DataToDataFrame(Columns, Data)

Please enter your query here: saiyan race


,animeTitle,animeDescription,URL
0,Dragon Ball Z,Five years after winning the World Martial Art...,https://myanimelist.net/anime/813/Dragon_Ball_Z
1,Dragon Ball Super: Broly,"Forty-one years ago on Planet Vegeta, home of ...",https://myanimelist.net/anime/36946/Dragon_Bal...
2,Dragon Ball Kai (Dragon Ball Z Kai),"Five years after the events of Dragon Ball, ma...",https://myanimelist.net/anime/6033/Dragon_Ball...
3,Dragon Ball Z Special 1: Tatta Hitori no Saish...,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...


## 2.2. Conjunctive query & Ranking score


For this part we are asking to given a query, show the top k documents that are more closer to this query using scoring schema.

To answer this question, first we should go through some definitions. 

### - What is tf?

We define tf(t, d) of term t in documnet d as the **division** of the **number of occurance of term t in document d** by **the total number of occurance of term t in all documents** in the corpus. You can see the formula here: 

### - What is idf?

Idf (inverse document frequency) is a measure of how much information the word provides. This value will be obtained by **dividing** the **total number of documents** by the **number of documents containing the term**, and then taking the **logarithm** of that quotient. You can see the formula here: 

### - What is tfidf?

Tfidf for each term and document can be obtained by the **multiplication** of **tf(t,d) and idf(t,D)**. The higher this value is, the more important that term is in that specific document. 

## 2.2.1 Creating the inverted indexes

Here we should build an inverted index in the format that each term corresponds to the documents in which that term is appearing and also the tfidf of that term in that specific document. 

Here we will define some functions to help us to do this process. 

### 2.2.1.1 Function tfidf

This function given the list of documents and the number of occurance of each specific term T in each of these documents and also the number of all the documents in the corpus, will return the terms and its tfidf for each of the documents that are containing that term. 

In [18]:
def GetTfIdf(invertedIndex, NumberOfDocs):
    for Term in invertedIndex: # For each term
        
        # How many times that term occured in the corpus
        TotalOccurance = sum(doc[1] for doc in invertedIndex[Term])
        
        # How many documents are containig that term 
        NumberOfTDocs = len(invertedIndex[Term])
        
        # Computing the idf of a term 
        idf = log(NumberOfDocs / NumberOfTDocs)
        
        for docs in invertedIndex[Term]: # For each document that is contaning the term 
            
            # Put the tfidf (tf*idf) instead of the number of occurence of that term in the document
            docs[1] = (docs[1] / TotalOccurance) * idf
            
    return invertedIndex

### 2.2.1.2 Create a suitable inverted index

Now we should write a code that build another type of inverted index, which each term corresponds to the documents that are contaning that term and also the number of occurence of the term in each of those documents. This inverted index will be saved in 'InvertedIndex_2.json' file.  

In [34]:
# To get the term_id of each term
Vocabularies = json.load(open("Vocabulary.json", "r"))

# To store the inverted index
# We will store our inverted index in this dictionary
# Keys: term
# Value: a list of (document_id (which is a pair of page and anime index), #occurance of the term in this document)
InvertedIndexes = defaultdict(set)

# For each page
for i in range(1, 384):
    # For each anime 
    for j in range((50 if i != 383 else 28)):
        
        # Read the '.csv' file for a specific anime
        Path = '/home/mehrdad/ADM-HW3/HTMLS/page'
        File = open(Path + str(i) + '/anime_' + str(j) +'_synopsisPrep.csv', mode = 'r')
        
        # Extract the words in the synopsis of that anime
        DescriptionWord = File.read().split(',')
        
        # Check the number of the occurances of each word in the synopsis of a specific anime
        NumberOfOccurenceInDoc = Counter(DescriptionWord)
        
        # For each term we will add this anime and its total number of occurance in this anime
        for term in NumberOfOccurenceInDoc:
            InvertedIndexes[Vocabularies[term]].add(((i,j), NumberOfOccurenceInDoc[term]))
        
        
        
# Sorting the document ids
for term in InvertedIndexes:
    InvertedIndexes[term] = sorted(InvertedIndexes[term])
    
# Store this inverted index in '.json' file to get the information pretty easily
with open("InvertedIndex_2.json", "w") as write_file:
    json.dump(InvertedIndexes, write_file)


Now we are able to compute the tf, idf and ea the end the tfidf of each term and docment. 

### 2.2.1.3 Main Creating the new  Inverted Index

Now here we will create a new inverted index such that each term has the information about each document that is containig that specific term and also the tfidf of that term and that specific document. This new inverted index will be saved in a file named **'InvertedIndex_3.json'**.

In [65]:
# Loading the second inverted index term -> documents, # Occurence
InvertedIndex = json.load(open("InvertedIndex_2.json", "r"))

# Number of the docuements in the corpus 
NumOfDocs = len(AllURLs) # Number of documents are the number of the anime

# Computing the tfidf for each term and document
NewInvertedIndex = GetTfIdf(InvertedIndex, NumOfDocs)

# Saving the new inverted index into a new file named 'InvertedIndex_3.json'
with open("InvertedIndex_3.json", "w") as write_file:
    json.dump(NewInvertedIndex, write_file)

### 2.2.1.4 Function query to tfidf

This function given the query, the second inverted index which each term corresonds to the docuemnts that are containig the number of occurence of that term in those documents, the vocabulary, and the number of all the docuemnts in the corpus will compute the tfidf of the term in the query. 

In [19]:
def QueryToTfidf(Query, invertedIndex,Vocabulary,  NumberOfDocs):
    
    
    # Mapping the words in query to their term_id
    PreProcessedQuery = StringPreProcess(Query)
    Query = [str(Vocabulary.get(Term)) for Term in PreProcessedQuery]
    
    # Check the number of occurence of each term in the query 
    QueryDict = Counter(Query)
    
    for Term in QueryDict:# for each term in the query
        
        # If that term is in the inverted index
        if invertedIndex.get(Term):

            # How many times that term occured in the corpus
            TotalOccurance = sum(doc[1] for doc in invertedIndex[Term])

            # How many documents are containig that term 
            NumberOfTDocs = len(invertedIndex[Term])

            # Computing the idf of a term 
            idf = log(NumberOfDocs / NumberOfTDocs)

            # Put the tfidf (tf*idf) instead of the number of occurence of that term in the query
            QueryDict[Term] = (QueryDict[Term]/ TotalOccurance) * idf
        else:
            QueryDict[Term] = 0
   
    return QueryDict

### 2.2.1.5 Function Dot Product 

In this function given two vectors, we will compute the dot product of the two and return back the result. 

In [20]:
def DotProduct(V1, V2): # The two given vectors should have the same length
    return sum(V1[i] * V2[i] for i in range(len(V1)))

### 2.2.1.6 Function Euclidean norm 

This function given a vector, will compute the Euclidean norm of that vector and return the result. 

In [21]:
# Computes the Eucledean norm the given vector
def Euclidean(Vector): 
    return (sum(Vector[i]**2 for i in range(len(Vector)))) ** .5

### 2.2.1.7 Function Query to Documents

In this function, given the query and the 3rd inverted index, we will extract the documents that are contaning all the terms in the query. The result will be a dictinoary which the keys are the documents and the keys are a list which is containing all the tfirdf of the terms in the query. 

In [22]:
def QueryToDocuemnts(Query, invertedIndex):
    
    AllTermsDict = []
    for term in Query: # For each term in the query
        TermDict = {} # The dictionary which contains the documents that are containing that term 
        
        # For each document which is contaning the term 
        if invertedIndex.get(term):
            for doc in invertedIndex[term]:

                # Add the document and the corresponded tfidf for that term in that document
                TermDict[tuple(doc[0])] = doc[1]
            # Add the dictionary for a specific term in the query which is containig all 
            # the document that are containig that term 

        AllTermsDict.append(TermDict.copy())
    
    # Extract the documents that are containing all the terms in the query
    
    # All the documents in each dictionary 
    AllDocs = [set(termDict.keys()) for termDict in AllTermsDict]
    
    AllIncludedDocs = AllDocs[0].intersection(*AllDocs) # Docs that are containing all the terms in the query
    
    # Here we are creating a dictionary which keys are the documents that are containig all the terms in the query
    # and the keys are the tfidf of all the term in the query in sequence. 
    DocsToTfidf = defaultdict(list)
    for termDict in AllTermsDict:
        for doc in AllIncludedDocs:
            DocsToTfidf[doc].append(termDict[doc])
    
    
    return DocsToTfidf

### 2.2.1.8 Function Similarity 

In this function given the query tfidf values and all the documents that are containig all the terms in the query, we want to compute the similarity of the query with each docuement.

* The cosine similarity between two vectores will be the **dot product of the values** in these two vectors, **divided** by **the multiplication of the euclidean norm** of each of the vectors. 

In [23]:
def Similarity(QueryTfidf, Documents):
    
    for doc in Documents:
        
        # Compute the dot product of tfidf of the terms in the query and a specific document 
        DotProductRes = DotProduct(QueryTfidf, Documents[doc])
        
        # Computer the similarity of query and a specific document 
        Similarity = DotProductRes/(Euclidean(QueryTfidf)*Euclidean(Documents[doc]))
        
        # Append the value of similarity at the end of the least of each document. 
        Documents[doc].append(Similarity)
    
    return Documents

* We are asked to sort the similar documents using heap sort. In order to use heap sort, we should have a class for the object that we are passing to the heap. 

### 2.2.1.9 Function Docs Class

In [24]:
class Docs:  
    
    # Storing the passed information for each object
    def __init__(self,DocInfo):  
        self.docInfo = DocInfo
        
    # Overriding the operator '<'. Here as we need the descending sort, we override in reverse. 
    def __lt__(self, other):  
        return self.docInfo[-1] > other.docInfo[-1]
   
    # Overriding the operator '>'
    def __gt__(self, other):  
        return other.__lt__(self)  

### 2.2.1.10 Function HeapSort

This function given a list of object will sort them by using heap tree and return the result. 

In [25]:
def heapsort(objects):  
    
    # Heap Tree
    heap = []  
    
    # Add all the given object to the tree
    for element in objects:  
        heappush(heap, element)  
   
    # Will store the ordered version of objects
    ordered = []  
    
    # Until we have still object in tree
    while heap:  
        #Take the root of the tree -> Which is the highest value. 
        ordered.append(heappop(heap))  
    
    return ordered  

### 2.2.1.10 Main function Ranking

In [26]:
# Get the query from the user
Query = input('Please enter your query here: ')

# First K most similar documents
K = int(input('Please enter the value of k here: '))

# Loading the second inverted index term -> documents, # Occurence
SecondInvertedIndex = json.load(open("InvertedIndex_2.json", "r"))

# Loading the third inverted index term -> documents, # tfidf
ThirdInvertedIndex = json.load(open("InvertedIndex_3.json", "r"))

# Loading the vocabulary file which maps each term to a specific ID
Vocabularies = json.load(open("Vocabulary.json", "r"))

# Compute the TFIDF of the query
QueryTFIDF = QueryToTfidf(Query, SecondInvertedIndex,Vocabularies, len(AllURLs))

# Take the documents that are containing all the words
RelatedDocuments = QueryToDocuemnts(QueryTFIDF.keys(), ThirdInvertedIndex)

# Compute the similarity of each document with query
DocumentsSimilarity = Similarity(list(QueryTFIDF.values()), deepcopy(RelatedDocuments))

# Here we will store the details of each similar documents 
Data = []
for Res in DocumentsSimilarity:
    Path = '/home/mehrdad/ADM-HW3/HTMLS/page'
    FileURL = open('URLs.txt', mode = 'r')
    FileInfo = open(Path + str(Res[0]) + '/anime_' + str(Res[1]) +'.tsv', mode = 'r')

    Information = FileInfo.read().split('\n')[1].split('\t')
    Title = Information[0]
    Description = Information[10]
    URL = FileURL.read().split('\n')[(Res[0]-1)*50 + Res[1]]
    Data.append([Title, Description, URL, DocumentsSimilarity[Res][-1]])
    
# Here we will sort the documents based on the similarity using heap tree
Data = [Doc.docInfo for Doc in heapsort([Docs(i) for i in Data])]

# Setting the similarity to 2 float places 
for doc in Data:
    doc[-1] = "{:.2f}".format(doc[-1])

# Here we will pick the first K documents that are more similar with respect to the given query
DataToDataFrame(['animeTitle','animeDescriptio', 'Url', 'Similarity'], Data[:K])


Please enter your query here: saiyan race
Please enter the value of k here: 10


,animeTitle,animeDescriptio,Url,Similarity
0,Dragon Ball Z,Five years after winning the World Martial Art...,https://myanimelist.net/anime/813/Dragon_Ball_Z,1.00
1,Dragon Ball Z Special 1: Tatta Hitori no Saish...,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...,1.00
2,Dragon Ball Kai (Dragon Ball Z Kai),"Five years after the events of Dragon Ball, ma...",https://myanimelist.net/anime/6033/Dragon_Ball...,1.00
3,Dragon Ball Super: Broly,"Forty-one years ago on Planet Vegeta, home of ...",https://myanimelist.net/anime/36946/Dragon_Bal...,1.00


# 3. Define a new score

In this question we let the user to include more information in his query. Most of the provided information will be working as a filter except the which user gives us for the synopsis. What we let the user include in his query and in which format: 

### Filters

1- **The type of anime:** If a user wants to include the type of the anime in the query, he should use this notation -> 1- {The type of anime}, ex. 1- tv

2- **The date of releasing:** If a user want to include a starting point which our search engine consider the anime which their release that is after that starting poin, he should use this notation -> 2- {released year}, ex. 2- 2009

3- **The date of ending:** If the user wants to include an end point in which our search engine not consider the anime that have been ended after this time then he should use this notation: 3- {end year}, ex. 3- 2020

4- **Some words in the synopsis:** If the user wants to include some words that those words has been occured in the resulted anime's synopsis, he should use this notation: 4- {words in synopsis}, ex. 4- saiyan race

* **Note:** We won't consider the documents that are missing some of the information provided in the query. 

### Scoring

To score the related documents and show them in order, we will consider these variables: 

**The normalized value for value x in a sequence of values X will be defined as (x - min(X))/(max(X) - min(X))**

1- **The cosine similarity:** The docuemnts that have a higher cosine similarity with respect to the words in synopsis given in the query, will have a higher priority.**To mention this value is between 1 and 0**

2- **Anime score and anime user:** We will consider a combination of anime score and its number of user. We will use a multiplication of the number of the users and the score of the anime. Then when we had this value for all the related animes, we will normalize the values according to its formula.

3- **Number of members:** As we know here the higher the member, the more popular the anime. So here we will normalize these values using the mentioned formula for normalization. 

**Final score: The final score of a document in our approach will be the sum of all of these values. The the ones that having higher score, will have a higher priority.**

* **Note:** If in any case we had some missing values, we will consider those values as 0. 

### *The main function for this part is at the end of this section*.

Now we go throguh the steps and the functions needed to build this search engine. 

## 1. Filtering

**Here we first filter the the documents with respect to the given query**

### 3.1.1 Function Query Process

In [27]:
# With the help of this function we will be able to extract each parameter in the query
def QueryProcess(Query):
    Strings = re.split('[1|2|3|4]-', Query.lower())[1:] 
    Numbers = re.findall('[1|2|3|4]+-', Query.lower())
    GivenConditions = ["".join(i) for i in zip(Numbers, Strings)]
    return GivenConditions

After getting the query, now we should filter the documents with respect to the given conditions. Here we will filter the by using 'Anime Type', 'Anime Released Data' and 'Anime End Data'. 

### 3.1.2 Function Filter Documents

In [28]:
def FilterDocs(Conditions):
    
    # This dictionary will contain the filtered documents and their necessary information
    FilteredDocs = {}
  
    # To have the URL of all the anime
    FileURL = open('URLs.txt', mode = 'r').read().split('\n')
    
    # For each page
    for page in range(1, 384):
        
        # For each anime
        for anime in range((50 if page != 383 else 28)):
            
            # Read the '.tsv' file of each specific anime
            Path = '/home/mehrdad/ADM-HW3/HTMLS/page'
            File = open(Path + str(page) + '/anime_' + str(anime) +'.tsv', mode = 'r')
            
            # Take the URL of a specific anime
            URL = FileURL[(page-1)*50 + anime]
            
            # The extracted information for that anime
            AnimeInfo = FilterInfo(File)
            
            # A flag which specifies if we should consider this docuemnt or it should be filtered
            Pick = True
            
            # For each condition that we have in the query
            for Con in Conditions:
                
                # Here will check each condition given in the query
                # Anime type
                # Releasing date
                # End date
                
                Con = list(map(str.strip, Con.split('-')))
                if Con[0] == '1' and AnimeInfo['animeType'] != Con[1]:
                    Pick = False
                elif (Con[0] == '2' and AnimeInfo['releaseDate'] < Con[1]) or not AnimeInfo['releaseDate'].isdigit():
                    Pick = False
                elif (Con[0] == '3' and (AnimeInfo['endDate'] > Con[1]) or not AnimeInfo['endDate'].isdigit()):
                    Pick = False 
            if Pick:
                AnimeInfo['URL'] = URL
                FilteredDocs[(page,anime)] = deepcopy(AnimeInfo)
    return FilteredDocs

### 3.1.3 Function Filter Necessary Information

In this function given a document we will extract the necessary information which will be 'Anime Title', 'Anime Score', 'Anime Users' and 'Anime members'. 

In [29]:
def FilterInfo(Document):
    
    # We will filter some useful information the given document
    ReadDocument = Document.read().split('\n')[1].split('\t')
    InfoDict = {'animeTitle':ReadDocument[0], 'animeScore': ReadDocument[6], 'animeUser': ReadDocument[7],
                'animeMembers': ReadDocument[5], 'releaseDate': ReadDocument[3].split(',')[-1].strip(), 
                'endDate':ReadDocument[4].split(',')[-1].strip(),'animeType': ReadDocument[1]}
    
    # We will lower case everything except Anime Title
    for i in list(InfoDict.keys())[1:]:
        InfoDict[i] = InfoDict[i].lower()
    
    return deepcopy(InfoDict)

## 3.1.4 Main Function Filter

Here we will get the query from the user in the format stated above.

In [30]:
def Filtering(Query):
    FilteredDocuments = FilterDocs(Query)

    # Then we should compute check the similarity between the query and the given filtered documents 
    # So we will add another property to each filtered data name synopsisSimilarity.
    # This value will be by default 0. 

    for Doc in FilteredDocuments:
        FilteredDocuments[Doc]['synopsisSimilarity'] = 0 

    # Then we will check the cosine similarity between the query and the synosis of the anime if this information
    # was provided in the query 
    
    SynopsisQuery = ''
    
    for q in Query:
        if '4-' in q:
            SynopsisQuery = q.split('- ')[-1]

    if SynopsisQuery:
        # Loading the second inverted index term -> documents, # Occurence
        SecondInvertedIndex = json.load(open("InvertedIndex_2.json", "r")) 
        
        # Loading the third inverted index term -> documents, # tfidf
        ThirdInvertedIndex = json.load(open("InvertedIndex_3.json", "r"))
                  
        # Loading the vocabulary file which maps each term to a specific ID
        Vocabularies = json.load(open("Vocabulary.json", "r"))

        # Compute the TFIDF of the query
        QueryTFIDF = QueryToTfidf(SynopsisQuery, SecondInvertedIndex,Vocabularies, len(AllURLs))

        # Take the documents that are containing all the words
        RelatedDocuments = QueryToDocuemnts(QueryTFIDF.keys(), ThirdInvertedIndex)

        # Compute the similarity of each document with query
        DocumentsSimilarity = Similarity(list(QueryTFIDF.values()), deepcopy(RelatedDocuments))
        # Here we will pick the similarity between our filtered data and given query
        for doc in FilteredDocuments:
            SimilarityOfDoc = DocumentsSimilarity.get(doc)
            if SimilarityOfDoc:
                FilteredDocuments[doc]['synopsisSimilarity'] = SimilarityOfDoc[-1]
    return FilteredDocuments

After having all the filtered documents and checking their similarity with respect to the given query, we will now add new scores to each documents. 

## Scoring

In the scoring part as we mentioned above, we will add new scores to each of the filtered documents. 

### 3.2.1 Function Score and User

This function, given all the filtered documents will consider a new score name 'animeSUScore' which is the multiplication of each 'animeScore' and 'animeUser'. At the end we will normalize the values.  

In [31]:
def RankUser(Docs):
    
    # Pick the default values for max and min 
    Min, Max = float("inf"), 0
    
    # For each document
    for doc in Docs: 
        
        # Try to multiply animeScore and animeUser
        try:
            Docs[doc]['animeSUScore'] = float(Docs[doc]['animeScore']) * float(Docs[doc]['animeUser'])
        
        # In case of an error we find out the values are missed or corrupted
        except: 
            
            # Consider the result of the multiplication 0
            Docs[doc]['animeSUScore'] = 0 
            
        # Update the min and max values
        Min, Max = min(Min, Docs[doc]['animeSUScore']), max(Max, Docs[doc]['animeSUScore'])
        
    # Normalize the values of animeSUScore
    for doc in Docs: 
        Docs[doc]['animeSUScore'] = ( Docs[doc]['animeSUScore'] - Min)/(Max - Min)

### 3.2.2 Function Member Score

This function given all the filtered documents will normalize the number of members of each document and consider that as a new score. 

In [32]:
def MemberScore(Docs):
    
    # Setting the default values for min and max
    Min, Max = float('inf'), 0
    
    # For each document
    for doc in Docs:
        
        # Try to conver the value of animeMembers to float
        try:
            Docs[doc]['animeMembers'] = float(Docs[doc]['animeMembers'])
            
        # If corrupted or missed 
        except: 
            
            # Set it to 0
            Docs[doc]['animeMembers'] = 0 
        
        # Update the min and max
        Min, Max = min(Min, Docs[doc]['animeMembers']), max(Max, Docs[doc]['animeMembers'])
    
    # Normalize the value of memberScore
    for doc in Docs:
        Docs[doc]['memberScore'] = (Docs[doc]['animeMembers'] - Min)/(Max - Min)

### 3.2.3 Function Final Score

This function given all the documents will compute the final score of each document which will be the sum of all the scores we just defined and also the cosine similarity.  

* **None:** As the cosine similarity between the query and the synopsis of the anime gives us more detail about the anime, we will give a coefficient of '3' to this value so that this value dominate the others.  

In [33]:
def FinalScore(Docs):
    # For each document we will compute the final score
    for doc in Docs:
        Docs[doc]['finalScore'] = Docs[doc]['memberScore'] + Docs[doc]['animeSUScore'] + 3 * Docs[doc]['synopsisSimilarity']

## 3.2.4 Main Score 

This function will go over all the filtered documents and compute the socre of each of the documents and show the documents in order of their final score. 

In [34]:
# We will give all the filtered documents to be scored
def Scoring(FilteredDocuments):
    RankUser(FilteredDocuments)
    MemberScore(FilteredDocuments)
    FinalScore(FilteredDocuments)
    return FilteredDocuments

## 3.3 Main Function Query to Documents

Here we take a query from the user and give back him the resulted documents with respect to the given query. 

**Important: Don't forget to use the format of the query when you want to issue a query"**

1- {Anime Type} 2- {Anime released date} 3- {Anime end date} 4- {Words in synopsis}<br/><br/>
**FYI:** You can issue a subset of the parameters in the query. 

In [35]:
# Processing the query and filter the related documents 
Query = QueryProcess(input("Please enter your query here: "))

# Filtering documents based on the given parameters in the query
FilteredDocuments = Filtering(Query)

# Scoring the filtered documents
ScoredDocuments = Scoring(FilteredDocuments)

# Here we will store the details of each related documents 
Data = []
for Res in ScoredDocuments:
    Path = '/home/mehrdad/ADM-HW3/HTMLS/page'
    FileInfo = open(Path + str(Res[0]) + '/anime_' + str(Res[1]) +'.tsv', mode = 'r')
    Description = FileInfo.read().split('\n')[1].split('\t')[10]

    Data.append([ScoredDocuments[Res]['animeTitle'], Description, ScoredDocuments[Res]['URL'],
                     ScoredDocuments[Res]['finalScore']])

Data = [Doc.docInfo for Doc in heapsort([Docs(i) for i in Data])]

# Setting the similarity to 2 float places 
for doc in Data:
    doc[-1] = "{:.2f}".format(doc[-1])
    
# Here we will pick the first K documents that are more similar with respect to the given query
DataToDataFrame(['animeTitle','animeDescription', 'Url', 'Final score'], Data[:10])

Please enter your query here: 1- tv 4- saiyan race


,animeTitle,animeDescription,Url,Final score
0,Dragon Ball Z,Five years after winning the World Martial Art...,https://myanimelist.net/anime/813/Dragon_Ball_Z,3.63
1,Dragon Ball Kai (Dragon Ball Z Kai),"Five years after the events of Dragon Ball, ma...",https://myanimelist.net/anime/6033/Dragon_Ball...,3.18
2,Death Note,"A shinigami, as a god of death, can kill any p...",https://myanimelist.net/anime/1535/Death_Note,2.00
3,Shingeki no Kyojin (Attack on Titan),"Centuries ago, mankind was slaughtered to near...",https://myanimelist.net/anime/16498/Shingeki_n...,1.99
4,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,https://myanimelist.net/anime/5114/Fullmetal_A...,1.64
5,One Punch Man,The seemingly ordinary and unimpressive Saitam...,https://myanimelist.net/anime/30276/One_Punch_Man,1.62
6,Sword Art Online,"In the year 2022, virtual reality has progress...",https://myanimelist.net/anime/11757/Sword_Art_...,1.51
7,Boku no Hero Academia (My Hero Academia),"The appearance of ""quirks,"" newly discovered s...",https://myanimelist.net/anime/31964/Boku_no_He...,1.46
8,Naruto,"Moments prior to Naruto Uzumaki's birth, a hug...",https://myanimelist.net/anime/20/Naruto,1.37
9,Tokyo Ghoul,Tokyo has become a cruel and merciless city—a ...,https://myanimelist.net/anime/22319/Tokyo_Ghoul,1.36


## 3.4 Comparing

In this section we will compare the results that we got from this search engine the previous search engine that we have been built in the section 2.2.

The search engine in part 3.3 is more powerful than the search engine in 2.2 due to these: 

- Search engine in 3.3 gives the user the oppurtunity to use a kind of filtering based on the type, released data and the end date of the anime, while we didn't have this feature in search engine 2.2.

- Search engine in 3.3 for the resulted anime that have the same cosine similarity will with the help of the other information of the anime can sort the documents with respect to their popularity and rank. The ones that are better will be higher in the result. 

- Search engine in 3.3 even if can not find any document that contains all the word given for the synopsis will give us the result based on the other parameters of filtering. 

- If for a query in synopsis we have less than K related docuements, the search engine in 3.3 can give the result based on the other parameters of the filtering. 

**In the end we discovered that our search engine in 3.3 works much better than the search engine in 2.2.**

# 5. Algorithmic question

## 5.1 Algorithm

Basically in this case we are trying to find the maximum of the elements of an array while these elements are not adjacent. Here we will explain how does our algorithm works. 

We can solve this quetion with a greedy algorithm which solves this question with time complexity of O(n) as we can compute this value by iterating over the values just once. Here you can find the algorithm:

At each step, we will consider two sums. First the maximum sum up to this iteration with the previous element included and another sum which is the maximum sum up to this iteration without the previous element. We call these two sums as 1-SumWithPrevious, 2- SumWithoutPreviuos

1- We go through the elements. 

2- As we assume that the previous element has been picked so we add the current element to the SumWithoutPrevious sum. 

3- We check if this value (SumWithoutPrevious) is greater than the value of SumWithPrevious or not. If this was the case then we will swap these two values as for the next item we have picked its adjacent item.

4- If at some point the value of SumWithoutPrevious + the current element was not greater than SumWithPrevious, it means that with or without the current element we had the maximum sum so far. As we didn't include the element in the sums so we won't have a problem for the next item to be considered. Here we can assign the SumWithoutPrevios to SumWithPrevious. 

5- At the end we will return the maximum of these two values, which will be store in SumWithPrevious. 

**You can check a detailed example in 5.3 section.**

## 5.2 Implementation

In [10]:
def MaximumSumWithoutAdjacent(List):
    # define two sums and necessary datastructures
    SumWithoutPrevious, SumWithPrevious = 0, 0
    
    # Showing the elements in the list
    print('The elements in the list: ', List)
    
    # Iterating over the elements
    for i in List:
        
        # Add the current value to the sum without previous
        SumWithoutPrevious += i
        
        # Check the condition 
        if SumWithoutPrevious > SumWithPrevious:
            
            # As we gain more score by picking the current element so we swap the values. 
            SumWithoutPrevious, SumWithPrevious = SumWithPrevious, SumWithoutPrevious
        
        # If by considering the current element, we don't gain more score 
        else:

            # Up to this point we have the maximum score by not picking the previous element
            SumWithoutPrevious = SumWithPrevious
            
    # Showing the last result. 
    print('\nResult:')
    print('\nMaximum sum without considering adjacent elements: \'', SumWithPrevious, '\'', sep = '')

### Give me an example here: 

In [361]:
MyList = list(map(int, input("Please give me the elements of your list: ").split()))
MaximumSumWithoutAdjacent(MyList)

Please give me the elements of your list: 30 40 25 50 30 20
The elements in the list:  [30, 40, 25, 50, 30, 20]

Result:

Maximum sum without considering adjacent elements: '110'


## 5.3 Steps of the algorithm

In [362]:
def MaximumSumWithoutAdjacentSteps(List):
    # define two sums and necessary datastructures
    SumWithoutPrevious, SumWithPrevious, NotPickLast = 0, 0, False
    
    # To keep track the picked elements in each sum 
    WithPrevious, WithoutPrevious = [], []
    
    # Showing the elements in the list
    print('The elements in the list: ', List)
    
    # Iterating over the elements
    for Index, i in enumerate(List):
        
        # To distinguish between the steps
        print('-' * 50)
        print('* Step '+str(Index+1)+' *\n')
        
        # Explaining of the procedure
        print('We are considering number \'', i, '\':', sep = '')
        print('\nCurrent value of SumWithPrevious:', SumWithPrevious)
        print('Current value of SumWithoutPrevious:', SumWithoutPrevious, end  = '\n\n')
        
        # Add the current value to the sum without previous
        SumWithoutPrevious += i
        print('Adding value \'', i, '\' to SumWithoutPrevious -> SumWithoutPrevious = ', SumWithoutPrevious, sep = '', end = '\n\n')
        
        # Check the condition 
        if SumWithoutPrevious > SumWithPrevious:
            
            # As we gain more score by picking the current element so we swap the values. 
            SumWithoutPrevious, SumWithPrevious = SumWithPrevious, SumWithoutPrevious
            
            # If we didn't pick the previous element
            if NotPickLast:
                print('We didn\'t pick the previous element!!!')
                
                # As we didn't pick the previous element, we add current value to with previous elements 
                WithPrevious.append(i)
                
            # If we have considered the previous element
            else:
                
                # If this is not the first iteration 
                if Index:
                    print('Swap two lists!!!')
                
                # Now we add current element to sumwithout previous set and swap two lists
                WithoutPrevious.append(i)
                WithPrevious, WithoutPrevious = WithoutPrevious[:], WithPrevious[:]
            
            # Showing the values at the current iteration 
            print('--->> Pick the current element!!!')
            print('\nSumWithoutPrevious is greater than SumWithPrevious -> Swap two maximum!')
            print('SumWithPrevious = ', SumWithPrevious, ' SumWithoutPrevious = ', SumWithoutPrevious)
            
            # We removed the previous element -> we didn't pick the previous element 
            NotPickLast = False
            
        # If by considering the current element, we don't gain more score 
        else:

            # Up to this point we have the maximum score by not picking the previous element
            SumWithoutPrevious = SumWithPrevious
            
            WithoutPrevious = WithPrevious[:]
            
            # We are not picking the current element, so for next iteration we didn't pick the previous one.
            NotPickLast = True
            
            # Showing the details of what we did exactly
            print('\n--->>Don\'t pick the element!!!')
            print('\nSumWithoutPrevious is not greater than SumWithPrevious -> Both take maximum!')
            print('SumWithPrevious = ', SumWithPrevious, ' SumWithoutPrevious = ', SumWithoutPrevious)
            
        # Showing the picked element up to this point. 
        print('\nMaximum sum with current element: ', WithPrevious)
        print('\nMaximum sum without current element: ', WithoutPrevious)
        
    
    # Showing the last result. 
    print('\n***************')
    print('Result:\n')
    print('All elements:', List)
    print('\nMaximum sum without considering adjacent elements: \'', SumWithPrevious, '\'', sep = '')
    print('By picking these values: ', WithPrevious)

### Run the detailed function here.

In [363]:
MaximumSumWithoutAdjacentSteps(MyList)

The elements in the list:  [30, 40, 25, 50, 30, 20]
--------------------------------------------------
* Step 1 *

We are considering number '30':

Current value of SumWithPrevious: 0
Current value of SumWithoutPrevious: 0

Adding value '30' to SumWithoutPrevious -> SumWithoutPrevious = 30

--->> Pick the current element!!!

SumWithoutPrevious is greater than SumWithPrevious -> Swap two maximum!
SumWithPrevious =  30  SumWithoutPrevious =  0

Maximum sum with current element:  [30]

Maximum sum without current element:  []
--------------------------------------------------
* Step 2 *

We are considering number '40':

Current value of SumWithPrevious: 30
Current value of SumWithoutPrevious: 0

Adding value '40' to SumWithoutPrevious -> SumWithoutPrevious = 40

Swap two lists!!!
--->> Pick the current element!!!

SumWithoutPrevious is greater than SumWithPrevious -> Swap two maximum!
SumWithPrevious =  40  SumWithoutPrevious =  30

Maximum sum with current element:  [40]

Maximum sum wit